# RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval

In [ ]:
# NOTE: An OpenAI API key must be set here for application initialization, even if not in use.
# If you're not utilizing OpenAI models, assign a placeholder string (e.g., "not_used").
import os
os.environ["OPENAI_API_KEY"] = "your-openai-key"

In [ ]:
import requests
from bs4 import BeautifulSoup

url="https://www.cs.umd.edu/class/spring2022/cmsc430/"
r = requests.get(url)

root=BeautifulSoup(r.content,"lxml")
lnks = root.findAll("a")


import os
from urllib.parse import urljoin
os.system('mkdir lectures')
outdir = 'lectures/'
for lnk in lnks:
    href=lnk['href']
    if href.lower().endswith(('.pdf','.py')):
        urld = urljoin(url, href)
        rd = requests.get(urld, stream=True)
        #Write the downloaded pdf to a file
        filename = urld.split("/")[-1]
        print(filename)
        outfile = os.path.join(outdir,filename)
        with open(outfile,'wb') as f:
            f.write(rd.content)


In [1]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

url = "https://www.cs.umd.edu/class/spring2022/cmsc430/Notes.html"
r = requests.get(url)

root = BeautifulSoup(r.content, "lxml")
lnks = root.findAll("a")

os.makedirs('html_files', exist_ok=True)  # Create the output directory
outdir = 'html_files/'

for lnk in lnks:
    href = lnk['href']
    if href.lower().endswith('.html') or href.lower().endswith('.htm'):  # Filter for HTML
        urld = urljoin(url, href)
        rd = requests.get(urld)

        # Write the downloaded HTML to a file
        filename = urld.split("/")[-1]
        if not filename:  # Handle cases where the filename is empty
            filename = 'index.html'  # Default filename
        outfile = os.path.join(outdir, filename)

        with open(outfile, 'w', encoding='utf-8') as f:  # Note the 'w' for text mode
            f.write(rd.text)  # .text gets the HTML content


KeyError: 'href'

In [7]:
import os
from bs4 import BeautifulSoup

def convert_html_to_text(input_dir, output_dir):
    """Converts HTML files in a directory to text files.

    Args:
        input_dir (str): Path to the directory containing HTML files.
        output_dir (str): Path to the directory where text files will be saved.
    """

    os.makedirs(output_dir, exist_ok=True)  # Create the output directory

    for filename in os.listdir(input_dir):
        if filename.endswith('.html') or filename.endswith('.htm'):
            filepath = os.path.join(input_dir, filename)
            with open(filepath, 'r',encoding='utf-8') as html_file:
                soup = BeautifulSoup(html_file, 'html.parser')

                # Get the text content
                text = soup.get_text(strip=True)

                # Create output filename (replace .html with .txt)
                output_filename = os.path.splitext(filename)[0] + '.txt'
                output_filepath = os.path.join(output_dir, output_filename)

                with open(output_filepath, 'w', encoding='utf-8') as text_file:
                    text_file.write(text)



In [16]:
import os

def append_text_files(input_dir, output_file):
	outdir = os.path.join(input_dir, output_file)
	print(outdir)
	with open(outdir, 'w', encoding='utf-8') as outfile:
		for filename in os.listdir(input_dir):
			if filename.endswith('.txt'):
				filepath = os.path.join(input_dir, filename)
				with open(filepath, 'r', encoding='utf-8') as infile:
					outfile.write(infile.read())
					outfile.write('\n')  # Add a newline between files


In [8]:
convert_html_to_text("html_files", "text_files")

In [17]:
append_text_files("text_files", "notes.txt")

text_files/notes.txt


In [18]:
# Cinderella story defined in sample.txt
with open('text_files/Loot.txt', 'r') as file:
    text = file.read()

print(text[:100])

17 Loot: lambda the ultimate►CMSC 430:Design and Implementation of Programming LanguagesSyllabusText


1) **Building**: RAPTOR recursively embeds, clusters, and summarizes chunks of text to construct a tree with varying levels of summarization from the bottom up. You can create a tree from the text in 'sample.txt' using `RA.add_documents(text)`.

2) **Querying**: At inference time, the RAPTOR model retrieves information from this tree, integrating data across lengthy documents at different abstraction levels. You can perform queries on the tree with `RA.answer_question`.

### Building the tree

In [19]:
from raptor import RetrievalAugmentation 

ModuleNotFoundError: No module named 'raptor'

In [ ]:
RA = RetrievalAugmentation()

# construct the tree
RA.add_documents(text)

### Querying from the tree

```python
question = # any question
RA.answer_question(question)
```

In [ ]:
question = "How did Cinderella reach her happy ending ?"

answer = RA.answer_question(question=question)

print("Answer: ", answer)

In [ ]:
# Save the tree by calling RA.save("path/to/save")
SAVE_PATH = "demo/cinderella"
RA.save(SAVE_PATH)

In [ ]:
# load back the tree by passing it into RetrievalAugmentation

RA = RetrievalAugmentation(tree=SAVE_PATH)

answer = RA.answer_question(question=question)
print("Answer: ", answer)

## Using other Open Source Models for Summarization/QA/Embeddings

If you want to use other models such as Llama or Mistral, you can very easily define your own models and use them with RAPTOR. 

In [ ]:
import torch
from raptor import BaseSummarizationModel, BaseQAModel, BaseEmbeddingModel, RetrievalAugmentationConfig
from transformers import AutoTokenizer, pipeline

In [ ]:
# if you want to use the Gemma, you will need to authenticate with HuggingFace, Skip this step, if you have the model already downloaded
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, pipeline
import torch

# You can define your own Summarization model by extending the base Summarization Class. 
class GEMMASummarizationModel(BaseSummarizationModel):
    def __init__(self, model_name="google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the GEMMA model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.summarization_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),  # Use "cpu" if CUDA is not available
        )

    def summarize(self, context, max_tokens=150):
        # Format the prompt for summarization
        messages=[
            {"role": "user", "content": f"Write a summary of the following, including as many key details as possible: {context}:"}
        ]
        
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the summary using the pipeline
        outputs = self.summarization_pipeline(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated summary
        summary = outputs[0]["generated_text"].strip()
        return summary


In [ ]:
class GEMMAQAModel(BaseQAModel):
    def __init__(self, model_name= "google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.qa_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
        )

    def answer_question(self, context, question):
        # Apply the chat template for the context and question
        messages=[
              {"role": "user", "content": f"Given Context: {context} Give the best full answer amongst the option to question {question}"}
        ]
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the answer using the pipeline
        outputs = self.qa_pipeline(
            prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated answer
        answer = outputs[0]["generated_text"][len(prompt):]
        return answer

In [ ]:
from sentence_transformers import SentenceTransformer
class SBertEmbeddingModel(BaseEmbeddingModel):
    def __init__(self, model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1"):
        self.model = SentenceTransformer(model_name)

    def create_embedding(self, text):
        return self.model.encode(text)


In [ ]:
RAC = RetrievalAugmentationConfig(summarization_model=GEMMASummarizationModel(), qa_model=GEMMAQAModel(), embedding_model=SBertEmbeddingModel())

In [ ]:
RA = RetrievalAugmentation(config=RAC)

In [ ]:
with open('demo/sample.txt', 'r') as file:
    text = file.read()
    
RA.add_documents(text)

In [ ]:
question = "How did Cinderella reach her happy ending?"

answer = RA.answer_question(question=question)

print("Answer: ", answer)